<a href="https://colab.research.google.com/github/geros3711/IBMDataScience/blob/main/Analyzing_Historical_Stock_Revenue_Data_and_Building_a_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Analyzing Historical Stock/Revenue Data and Building a Dashboard

In [1]:
!pip install yfinance

In [24]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [25]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [37]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2024--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2024-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [28]:
tesla= yf.Ticker("TSLA")

In [29]:
tesla_data = tesla.history(period="max")

In [30]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [31]:
# prompt: read text file from path
# reading from https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue directly is not allowed
# I dowloaded contents manually to 'revenue.html' and webscrape from there
filename = 'revenue.html'
with open(filename, 'r') as f:
  html_data = f.read()
soup = BeautifulSoup(html_data, 'html5lib')
soup.title

<title>Tesla Revenue 2010-2023 | TSLA | MacroTrends</title>

In [32]:
read_html_pandas_data = pd.read_html(str(soup), match="Tesla Quarterly Revenue")
tesla_revenue = read_html_pandas_data[0]
tesla_revenue.columns = ['Date', 'Revenue']
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"")
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue.tail()

,Date,Revenue
53,2010-09-30,31
54,2010-06-30,28
55,2010-03-31,21
57,2009-09-30,46
58,2009-06-30,27


In [33]:
gme= yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [34]:
# prompt: read text file from path
# reading from https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue directly is not allowed
# I dowloaded contents manually to 'gme_revenue.html' and webscrape from there
filename = 'gme_revenue.html'
with open(filename, 'r') as f:
  html_data = f.read()
soup = BeautifulSoup(html_data, 'html5lib')
soup.title

<title>GameStop Revenue 2010-2023 | GME | MacroTrends</title>

In [35]:
read_html_pandas_data = pd.read_html(str(soup), match="Quarterly Revenue")
gme_revenue = read_html_pandas_data[0]
gme_revenue.columns = ['Date', 'Revenue']
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"")
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]
gme_revenue.tail()

,Date,Revenue
55,2010-01-31,3524
56,2009-10-31,1835
57,2009-07-31,1739
58,2009-04-30,1981
59,2009-01-31,3492


In [38]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

In [39]:
make_graph(gme_data, gme_revenue, 'GME')